# Evaluation of MLP log-likelihood for multiclass classification

Confirm PyTorch and manually coded MLP log-likelihood coincide

In [1]:
## Import packages

import torch
import torch.nn as nn

from eeyore.models.mlp import Hyperparameters, MLP
from eeyore.data import Iris

## Compute MLP log-likelihood using eeyore API version

In [2]:
## Load iris data

iris = Iris()

data = iris.data
labels = iris.labels

In [3]:
## Setup MLP model

hparams = Hyperparameters(dims=[4, 3, 3], activations=[torch.sigmoid, None])
model = MLP(hparams=hparams, loss=lambda x, y: nn.CrossEntropyLoss(reduction='sum')(x, torch.argmax(y, 1)), dtype=torch.float64)

In [4]:
## Fix model parameters

# theta = torch.rand(27, dtype=torch.float64)
theta = torch.tensor([
    0.7735, 0.8161, 0.3910, 0.9622, 0.3748, 0.8711, 0.3315, 0.5473, 0.8820,
    0.0294, 0.9686, 0.8313, 0.6693, 0.8791, 0.6271, 0.8636, 0.3814, 0.0319,
    0.5148, 0.5086, 0.7428, 0.5464, 0.5278, 0.6127, 0.4499, 0.1538, 0.9291], dtype=torch.float64)
model.set_params(theta.clone().detach())

In [5]:
## Compute MLP log-likelihood using eeyore API version

result01 = model.log_lik(data, labels)
result01

tensor(-176.2545, dtype=torch.float64, grad_fn=<NegBackward>)

In [6]:
## Compute MLP log-likelihood using eeyore loss method in MLP model

result02 = -model.loss(model(data), labels)
result02

tensor(-176.2545, dtype=torch.float64, grad_fn=<NegBackward>)

## Compute MLP log-likelihood using Pytorch loss and cross entropy

In [7]:
result03 = -nn.CrossEntropyLoss(reduction='sum')(model(data), torch.argmax(labels, 1))
result03

tensor(-176.2545, dtype=torch.float64, grad_fn=<NegBackward>)

In [8]:
result04 = -nn.NLLLoss(reduction='sum')(nn.Softmax(dim=1)(model(data)).log(), torch.argmax(labels, 1))
result04

tensor(-176.2545, dtype=torch.float64, grad_fn=<NegBackward>)

## Compute MLP log-likelihood manually

In [9]:
def cross_entropy_loss(data, labels):
    n = labels.size(dim=0)
    
    logit = model(data)
    
    softmax_vals = nn.Softmax(dim=1)(logit).log()
    labels_argmax = torch.argmax(labels, 1)
    
    result = 0
    for i in range(n):
        result = result + softmax_vals[i, labels_argmax[i]]
        
    return result

In [10]:
result05 = cross_entropy_loss(data, labels)
result05

tensor(-176.2545, dtype=torch.float64, grad_fn=<AddBackward0>)

## Print out values of all log-lik implementations

In [11]:
[p.data.item() for p in [result01, result02, result03, result04, result05]]

[-176.2544991888255,
 -176.2544991888255,
 -176.2544991888255,
 -176.25449918882543,
 -176.25449918882543]